### Starting from scratch on Covid-19 data analysis

In [9]:
import geopandas as gpd

In [2]:
import numpy as np
import pandas as pd

In [3]:
import sys

In [4]:
sys.path

['/Users/khazelton/opt/conda/covidz/notebooks',
 '/Users/khazelton/opt/anaconda3/envs/covidz/lib/python38.zip',
 '/Users/khazelton/opt/anaconda3/envs/covidz/lib/python3.8',
 '/Users/khazelton/opt/anaconda3/envs/covidz/lib/python3.8/lib-dynload',
 '',
 '/Users/khazelton/opt/anaconda3/envs/covidz/lib/python3.8/site-packages',
 '/Users/khazelton/opt/anaconda3/envs/covidz/lib/python3.8/site-packages/IPython/extensions',
 '/Users/khazelton/.ipython']

In [10]:
import seaborn as sns

In [7]:
import folium

In [12]:
import branca.colormap as cm

In [14]:
import matplotlib.pyplot as plt

In [15]:
from pathlib import Path

In [34]:
projdir = Path.cwd().parent
if str(projdir) not in sys.path:
    sys.path.append(str(projdir))
# from src.common import loadenv as const

In [35]:
projdir

PosixPath('/Users/khazelton/opt/conda/covidz')

In [36]:
fips_fn = projdir / 'data' / 'raw' / 'fips' / 'all-geocodes-v2018.xlsx'

In [37]:
fips_fn

PosixPath('/Users/khazelton/opt/conda/covidz/data/raw/fips/all-geocodes-v2018.xlsx')

In [46]:
path_str = fips_fn.as_posix()

In [47]:
path_str

'/Users/khazelton/opt/conda/covidz/data/raw/fips/all-geocodes-v2018.xlsx'

### Read an Excel file into a pandas DataFrame.

In [87]:
dff = pd.read_excel(path_str, skiprows=4)

In [88]:
dff.shape

(43847, 7)

In [89]:
dff

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,0,0,0,0,United States
1,40,1,0,0,0,0,Alabama
2,50,1,1,0,0,0,Autauga County
3,50,1,3,0,0,0,Baldwin County
4,50,1,5,0,0,0,Barbour County
...,...,...,...,...,...,...,...
43842,50,72,145,0,0,0,Vega Baja Municipio
43843,50,72,147,0,0,0,Vieques Municipio
43844,50,72,149,0,0,0,Villalba Municipio
43845,50,72,151,0,0,0,Yabucoa Municipio


In [90]:
dff.info

<bound method DataFrame.info of        Summary Level  State Code (FIPS)  County Code (FIPS)  \
0                 10                  0                   0   
1                 40                  1                   0   
2                 50                  1                   1   
3                 50                  1                   3   
4                 50                  1                   5   
...              ...                ...                 ...   
43842             50                 72                 145   
43843             50                 72                 147   
43844             50                 72                 149   
43845             50                 72                 151   
43846             50                 72                 153   

       County Subdivision Code (FIPS)  Place Code (FIPS)  \
0                                   0                  0   
1                                   0                  0   
2                              

### Import county population from census bureau data

In [94]:
cp_fn = projdir / 'data' / 'raw' / 'censusBurPop' / 'co-est2019-alldata.csv'

In [95]:
cp_fn

PosixPath('/Users/khazelton/opt/conda/covidz/data/raw/censusBurPop/co-est2019-alldata.csv')

In [131]:
path_str = cp_fn.as_posix()

In [132]:
path_str

'/Users/khazelton/opt/conda/covidz/data/raw/censusBurPop/co-est2019-alldata.csv'

In [133]:
cp_df = pd.read_csv(path_str, dtype={'STATE': float}, encoding='ISO-8859–1')

In [135]:
cp_df.dtypes

SUMLEV           int64
REGION           int64
DIVISION         int64
STATE          float64
COUNTY           int64
                ...   
RNETMIG2015    float64
RNETMIG2016    float64
RNETMIG2017    float64
RNETMIG2018    float64
RNETMIG2019    float64
Length: 164, dtype: object

In [134]:
cp_df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,40,3,6,1.0,0,Alabama,Alabama,4779736,4780125,4785437,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
1,50,3,6,1.0,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1.0,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1.0,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1.0,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56.0,37,Wyoming,Sweetwater County,43806,43806,43574,...,-17.605427,0.502513,15.035735,-4.614071,-13.140225,-13.762031,-18.484164,-23.515727,-18.743773,-17.065527
3189,50,4,8,56.0,39,Wyoming,Teton County,21294,21298,21296,...,-1.583464,-2.575509,1.905293,23.259519,12.508872,4.670450,1.253214,-0.171608,-12.432212,1.797445
3190,50,4,8,56.0,41,Wyoming,Uinta County,21118,21121,21089,...,-9.181105,-18.196975,-4.487730,-10.870861,-15.033634,-10.484550,-11.485101,-18.923455,-13.554993,-9.181105
3191,50,4,8,56.0,43,Wyoming,Washakie County,8533,8528,8530,...,-7.652085,-12.839390,-3.084589,-1.307811,-18.938032,0.000000,-15.204038,-15.950541,-17.246806,-8.289759


In [102]:
cp_df.shape

(3193, 164)

In [101]:
for col in cp_df.columns: 
    print(col) 

SUMLEV
REGION
DIVISION
STATE
COUNTY
STNAME
CTYNAME
CENSUS2010POP
ESTIMATESBASE2010
POPESTIMATE2010
POPESTIMATE2011
POPESTIMATE2012
POPESTIMATE2013
POPESTIMATE2014
POPESTIMATE2015
POPESTIMATE2016
POPESTIMATE2017
POPESTIMATE2018
POPESTIMATE2019
NPOPCHG_2010
NPOPCHG_2011
NPOPCHG_2012
NPOPCHG_2013
NPOPCHG_2014
NPOPCHG_2015
NPOPCHG_2016
NPOPCHG_2017
NPOPCHG_2018
NPOPCHG_2019
BIRTHS2010
BIRTHS2011
BIRTHS2012
BIRTHS2013
BIRTHS2014
BIRTHS2015
BIRTHS2016
BIRTHS2017
BIRTHS2018
BIRTHS2019
DEATHS2010
DEATHS2011
DEATHS2012
DEATHS2013
DEATHS2014
DEATHS2015
DEATHS2016
DEATHS2017
DEATHS2018
DEATHS2019
NATURALINC2010
NATURALINC2011
NATURALINC2012
NATURALINC2013
NATURALINC2014
NATURALINC2015
NATURALINC2016
NATURALINC2017
NATURALINC2018
NATURALINC2019
INTERNATIONALMIG2010
INTERNATIONALMIG2011
INTERNATIONALMIG2012
INTERNATIONALMIG2013
INTERNATIONALMIG2014
INTERNATIONALMIG2015
INTERNATIONALMIG2016
INTERNATIONALMIG2017
INTERNATIONALMIG2018
INTERNATIONALMIG2019
DOMESTICMIG2010
DOMESTICMIG2011
DOMESTICMIG2012

### import covid-19 stats

In [104]:
cvd_fn = projdir / 'data' / 'raw' / 'covid-19-data' / 'nyt-covid-19-data-master-us-counties.csv'

In [105]:
cvd_fn

PosixPath('/Users/khazelton/opt/conda/covidz/data/raw/covid-19-data/nyt-covid-19-data-master-us-counties.csv')

In [147]:
path_str = cvd_fn.as_posix()

In [148]:
path_str

'/Users/khazelton/opt/conda/covidz/data/raw/covid-19-data/nyt-covid-19-data-master-us-counties.csv'

In [139]:
cvd_df = pd.read_csv(path_str, )

In [109]:
cvd_df.shape

(33251, 6)

In [110]:
cvd_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
...,...,...,...,...,...,...
33246,2020-04-04,Sublette,Wyoming,56035.0,1,0
33247,2020-04-04,Sweetwater,Wyoming,56037.0,4,0
33248,2020-04-04,Teton,Wyoming,56039.0,36,0
33249,2020-04-04,Uinta,Wyoming,56041.0,2,0


In [140]:
dff.dtypes

Summary Level                                                int64
State Code (FIPS)                                            int64
County Code (FIPS)                                           int64
County Subdivision Code (FIPS)                               int64
Place Code (FIPS)                                            int64
Consolidtated City Code (FIPS)                               int64
Area Name (including legal/statistical area description)    object
dtype: object

In [141]:
cvd_df.dtypes

date       object
county     object
state      object
fips      float64
cases       int64
deaths      int64
dtype: object

In [151]:
cvd_df = pd.read_csv(path_str, parse_dates=['date'], dtype={'fips':str})

In [152]:
cvd_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
33246,2020-04-04,Sublette,Wyoming,56035,1,0
33247,2020-04-04,Sweetwater,Wyoming,56037,4,0
33248,2020-04-04,Teton,Wyoming,56039,36,0
33249,2020-04-04,Uinta,Wyoming,56041,2,0


In [153]:
cvd_df.shape

(33251, 6)

In [154]:
cvd_df.dtypes

date      datetime64[ns]
county            object
state             object
fips              object
cases              int64
deaths             int64
dtype: object

### Import county shape files

In [155]:
ctyshp_fp = projdir / 'data' / 'raw' / 'tl_2019_us_county' / 'tl_2019_us_county.shp'

In [156]:
ctyshp_str = ctyshp_fp.as_posix()

In [157]:
ctyshp_str

'/Users/khazelton/opt/conda/covidz/data/raw/tl_2019_us_county/tl_2019_us_county.shp'

In [159]:
cty_dropcols = [ 'STATEFP',
                  # 'COUNTYFP',
                   'COUNTYNS',
                  # 'GEOID',
                  # 'NAME',
                   'NAMELSAD',
                   'LSAD',
                   'CLASSFP',
                   'MTFCC',
                   'CSAFP',
                   'CBSAFP',
                  # 'METDIVFP',
                   'FUNCSTAT',
                   'ALAND',
                   'AWATER',
                   'INTPTLAT',
                   'INTPTLON'
                  # 'geometry'
                ]


In [160]:
cty_dropcols

['STATEFP',
 'COUNTYNS',
 'NAMELSAD',
 'LSAD',
 'CLASSFP',
 'MTFCC',
 'CSAFP',
 'CBSAFP',
 'FUNCSTAT',
 'ALAND',
 'AWATER',
 'INTPTLAT',
 'INTPTLON']

In [161]:
cty_gdf = gpd.read_file(ctyshp_fp)

In [163]:
cty_gdf.shape

(3233, 18)

In [164]:
cty_gdf.drop(columns=cty_dropcols, inplace=True)

In [165]:
cty_gdf.shape

(3233, 5)

In [196]:
cty_gdf.dtypes

COUNTYFP      object
GEOID         object
NAME          object
METDIVFP      object
geometry    geometry
dtype: object

In [166]:
cty_gdf

,COUNTYFP,GEOID,NAME,METDIVFP,geometry
0,039,31039,Cuming,None,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,069,53069,Wahkiakum,None,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,011,35011,De Baca,None,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,109,31109,Lancaster,None,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,129,31129,Nuckolls,None,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."
...,...,...,...,...,...
3228,123,13123,Gilmer,None,"POLYGON ((-84.65478 34.66559, -84.65488 34.669..."
3229,135,27135,Roseau,None,"POLYGON ((-96.40466 48.80528, -96.40467 48.813..."
3230,089,28089,Madison,None,"POLYGON ((-90.09363 32.70763, -90.09360 32.707..."
3231,227,48227,Howard,None,"POLYGON ((-101.69227 32.27106, -101.69221 32.2..."


In [203]:
cty_gdf = cty_gdf.rename(columns={'GEOID': 'fips'})

In [168]:
unknowns_df = cvd_df.loc[cvd_df['county'].str.contains('Unknown')].copy()

In [169]:
cvd_df.drop(unknowns_df.index, inplace=True)

In [171]:
cvd_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
33246,2020-04-04,Sublette,Wyoming,56035,1,0
33247,2020-04-04,Sweetwater,Wyoming,56037,4,0
33248,2020-04-04,Teton,Wyoming,56039,36,0
33249,2020-04-04,Uinta,Wyoming,56041,2,0


In [172]:
cvd_nofips_df = cvd_df.loc[cvd_df['fips'].isnull()].copy()

In [173]:
cvd_nofips_df.shape

(51, 6)

In [174]:
cvd_nofips_df

,date,county,state,fips,cases,deaths
416,2020-03-01,New York City,New York,NaN,1,0
448,2020-03-02,New York City,New York,NaN,1,0
482,2020-03-03,New York City,New York,NaN,2,0
518,2020-03-04,New York City,New York,NaN,2,0
565,2020-03-05,New York City,New York,NaN,4,0
627,2020-03-06,New York City,New York,NaN,5,0
715,2020-03-07,New York City,New York,NaN,12,0
820,2020-03-08,New York City,New York,NaN,14,0
947,2020-03-09,New York City,New York,NaN,20,0
1098,2020-03-10,New York City,New York,NaN,37,0


### set fips for NYC to '1', Kansas City to '2' since they are NAN in original data set

In [175]:
cvd_df.loc[cvd_df['county'] == 'New York City', 'fips'] = '1'
cvd_df.loc[(cvd_df['county'] == 'Kansas City') & (cvd_df['state'] == 'Missouri'), 'fips'] = '2'

In [178]:
cvd_df.loc[cvd_df['county'] == 'New York City']

,date,county,state,fips,cases,deaths
416,2020-03-01,New York City,New York,1,1,0
448,2020-03-02,New York City,New York,1,1,0
482,2020-03-03,New York City,New York,1,2,0
518,2020-03-04,New York City,New York,1,2,0
565,2020-03-05,New York City,New York,1,4,0
627,2020-03-06,New York City,New York,1,5,0
715,2020-03-07,New York City,New York,1,12,0
820,2020-03-08,New York City,New York,1,14,0
947,2020-03-09,New York City,New York,1,20,0
1098,2020-03-10,New York City,New York,1,37,0


In [183]:
cvd_df.loc[(cvd_df['county'] == 'Kansas City') & (cvd_df['state'] == 'Missouri')]

,date,county,state,fips,cases,deaths
5641,2020-03-20,Kansas City,Missouri,2,12,0
6631,2020-03-21,Kansas City,Missouri,2,13,0
7720,2020-03-22,Kansas City,Missouri,2,13,0
8908,2020-03-23,Kansas City,Missouri,2,18,0
10213,2020-03-24,Kansas City,Missouri,2,30,0
11640,2020-03-25,Kansas City,Missouri,2,51,0
13246,2020-03-26,Kansas City,Missouri,2,64,0
14981,2020-03-27,Kansas City,Missouri,2,78,0
16815,2020-03-28,Kansas City,Missouri,2,94,0
18749,2020-03-29,Kansas City,Missouri,2,102,0


### merge corona virus county data set with county shape files

In [214]:
comb_df = cty_gdf.merge(cvd_df, on='fips')

In [215]:
comb_df.shape

(32764, 10)

In [216]:
comb_df.dtypes

COUNTYFP            object
fips                object
NAME                object
METDIVFP            object
geometry          geometry
date        datetime64[ns]
county              object
state               object
cases                int64
deaths               int64
dtype: object

In [217]:
comb_df

,COUNTYFP,fips,NAME,METDIVFP,geometry,date,county,state,cases,deaths
0,039,31039,Cuming,None,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",2020-04-01,Cuming,Nebraska,1,0
1,039,31039,Cuming,None,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",2020-04-02,Cuming,Nebraska,1,0
2,039,31039,Cuming,None,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",2020-04-03,Cuming,Nebraska,1,0
3,039,31039,Cuming,None,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",2020-04-04,Cuming,Nebraska,1,0
4,069,53069,Wahkiakum,None,"POLYGON ((-123.43639 46.23820, -123.44759 46.2...",2020-04-04,Wahkiakum,Washington,1,0
...,...,...,...,...,...,...,...,...,...,...
32759,089,28089,Madison,None,"POLYGON ((-90.09363 32.70763, -90.09360 32.707...",2020-03-31,Madison,Mississippi,42,0
32760,089,28089,Madison,None,"POLYGON ((-90.09363 32.70763, -90.09360 32.707...",2020-04-01,Madison,Mississippi,53,0
32761,089,28089,Madison,None,"POLYGON ((-90.09363 32.70763, -90.09360 32.707...",2020-04-02,Madison,Mississippi,55,0
32762,089,28089,Madison,None,"POLYGON ((-90.09363 32.70763, -90.09360 32.707...",2020-04-03,Madison,Mississippi,65,0
